# Decision Trees for Rulesetting

We will use a decision tree to determine if the prior data collected in section one is within a specified ruleset and in or out of the bag.

While many ML algorithms are considered 'black boxes' a decision tree is much more human readable and allows for us to better determine why it predicts or acts in the way that it does.

This exercise falls within the use of supervised machine learning and is using a model we will train on training data to make predicitons.

In [1]:
data = [['a', 0, 'good'], ['a', 101, 'good'], ['b', -1, 'bad']] #['letter', 'number', 'class']

you can see how if you were to classify the 3 data points above as either good or bad, that you could choose to say if letter == a then good or you could choose if number > -1 then good, else bad.

Either way this is how decision trees work by splitting hte data into subgroups and using the features provided (like letter and number) to make a prediction on class.

As data sets grow larger and features become more plentiful you will need better methods for splittign the tree and accouting for the increase in entropy (the measure of uncertainty) in the model.

We can also represent trees with dictionaries shown below.

In [2]:
tree = {'letter': {'a': 'good', 'b': 'bad'}, 'number':{0: 'good', 101: 'good', -1: 'bad'}}

In [3]:
tree['letter']['a']

'good'

In [4]:
tree['letter']['b']

'bad'

In [9]:
tree['number'][-1] #can replace with 0 or -1; anything else will return a key error since we don't use conditionals yet

'bad'

Now we can sort the data points into groups and push the data point on to the next decision in the tree. 

Lets first set up a partition over numbers, then a second grouping based on letter to place these datum (singular points of data) into thier final groupings as good or bad.

In [10]:
good_group = []
bad_group = []

for key, value in tree.items():
    if key == 'number':
        for k,v in value.items():
            if k > -1:
                good_group.append(k)
            else:
                bad_group.append(k)
    if key == 'letter':
        for k,v in value.items():
            if k == 'a':
                good_group.append(k)
            else:
                bad_group.append(k)
        

In [11]:
print(good_group,bad_group)

['a', 0, 101] ['b', -1]


To figure out the entropy (uncertainty) of our dataset, we use the proportion of our data within each category or feature.

 $ H_{(entropy)} = - \sum^n_{i=1} P(X_i) * log_2 P(X_i)  $

Log base 2 provides us a range between 0 and 1.

Now that we have a basis for understanding entropy and calculating split points, we can attempt to apply this to our data we pickled from section 1.

In [12]:
import pickle

with open('../Section 1 - Code a Turtle Out of a Bag/data_rand', 'rb') as f:
    L = pickle.load(f)

In [13]:
L #[X,Y,Out of the box or not]

[[0.0, 0.0, False],
 [-5.485152762143779, 5.114987700468738, False],
 [-16.455458286431337, 15.344963101406211, False],
 [-32.91091657286268, 30.689926202812416, False],
 [-54.85152762143782, 51.149877004687355, True],
 [0.0, 0.0, False],
 [-6.797308402774876, -3.1696369630552446, False],
 [-19.084589067109754, 5.434009582210445, False],
 [-17.12358485528745, 27.848390289274718, False],
 [10.065648755812045, 40.526938141495705, True],
 [0.0, 0.0, False],
 [-1.5593376811331934, 7.336107005503543, False],
 [13.112876329873892, 10.45478236776993, False],
 [17.790889373273473, -11.553538648740698, False],
 [-11.553538648740698, -17.790889373273476, False],
 [-19.350227054406673, 18.889645654244237, False],
 [24.666414978614583, 28.24567174104341, False],
 [35.58177874654695, -23.10707729748139, False],
 [-23.10707729748139, -35.58177874654696, False],
 [-37.14111642768016, 30.443184302984932, True],
 [0.0, 0.0, False],
 [0.13089304827962348, -7.498857713672934, False],
 [-14.86682237906624

In [14]:
def escaped_turtles(data):
    turtle_list = []
    for i in range(len(data)):
        if data[i][-1] == True:
            turtle_list.append(data[i][:-1])
    return turtle_list

In [15]:
import collections
import math
import operator

def entropy(data):
    frequency = collections.Counter([item[-1] for item in data]) # output == ({False: 73, True: 10})
    def item_entropy(category):
        ratio = float(category)/len(data) #ratio of category to len of dataset
        return -1 * ratio * math.log2(ratio) #neg log base 2 of this item
    return sum(item_entropy(c) for c in frequency.values()) #sum it all up to return entropy

In [16]:
print(entropy(L))

0.5762914612174369


In [17]:
def best_feature_for_split(data):
    baseline = entropy(data)
    def feature_entropy(f):
        def e(v):
            partitioned_data = [d for d in data if d[f] == v]
            proportion = (float(len(partitioned_data)) / float(len(data)))
            return proportion * entropy(partitioned_data)
        return sum(e(v) for v in set([d[f] for d in data]))
    features = len(data[0]) - 1
    informaiton_gain = [baseline - feature_entropy(f) for f in range(features)]
    best_feature, best_gain = max(enumerate(informaiton_gain), key=operator.itemgetter(1))
    return best_feature

In [18]:
best_feature_for_split(L)

0

In [19]:
def potential_leaf_node(data):
    '''
    returns a tuple of the most common category and a count (category, count)
    '''
    count = collections.Counter([i[-1] for i in data])
    return count.most_common(1)[0]

In [20]:
potential_leaf_node(L)

(False, 63)

In [21]:
def create_tree(data, label):
    category, count = potential_leaf_node(data)
    if count == len(data):
        return category
    node = {}
    feature = best_feature_for_split(data)
    feature_label = label[feature]
    node[feature_label] = {}
    classes = set([d[feature] for d in data])
    for c  in classes:
        partitioned_data = [d for d in data if d[feature] == c]
        node[feature_label][c] = create_tree(partitioned_data, label)
    return node

In [22]:
def classify(tree, label, data):
    root = list(tree.keys())[0]
    node = tree[root]
    index = label.index(root)
    for k in node.keys():
        if data[index] == k:
            if isinstance(node[k],dict):
                return classify(node[k], label, data)
            else:
                return node[k]

In [23]:
def as_rule_str(tree, label, ident=0):
    space_ident = '  ' * ident
    s = space_ident
    root = list(tree.keys())[0]
    node = tree[root]
    index = label.index(root)
    for k in node.keys():
        s += 'if ' + label[index] + ' = ' + str(k)
        if isinstance(node[k], dict):
            s += ':\n' + space_ident + as_rule_str(node[k], label, idnet + 1)
        else:
            s += ' then ' + str(node[k]) + ('.\n' if ident == 0 else ', ')
    if s[-2:] == ', ':
        s = s[:2]
    s+= '\n'
    return s

In [24]:
data = [[0,0, False], [1,0,False], [0,1,True], [1,1,True]]
label = ['x','y','out']

In [25]:
tree =  create_tree(data, label)

In [26]:
print(as_rule_str(tree, label))

if y = 0 then False.
if y = 1 then True.




In [27]:
print(classify(tree, label, [1,1]))
print(classify(tree, label, [0,0]))
print(classify(tree, label, [1,2])) # cant classify what it hasn't seen
print(classify(tree, label, [3,4])) # cant classify what it hasn't seen
print(classify(tree, label, [-1,-1])) # cant classify what it hasn't seen

True
False
None
None
None


In [28]:
tree = create_tree(L, label)

In [29]:
print(as_rule_str(tree, label))

if x = 0.0 then False.
if x = 0.7839634745073975 then False.
if x = 1.0437982572004927 then False.
if x = 1.376527000978303 then False.
if x = 0.9140200755386061 then False.
if x = 6.143640332167439 then False.
if x = 6.086693480921191 then False.
if x = 10.065648755812045 then True.
if x = 11.463673814085455 then False.
if x = 12.287280664334876 then False.
if x = 13.112876329873892 then False.
if x = 14.73592933078662 then False.
if x = 15.390394572184746 then False.
if x = 15.638427827253564 then False.
if x = 17.790889373273473 then False.
if x = 18.43092099650231 then False.
if x = 29.119699051280804 then True.
if x = 23.70587799431179 then False.
if x = 24.666414978614583 then False.
if x = 24.574561328669738 then False.
if x = 26.546644282796144 then False.
if x = 28.409658841356187 then False.
if x = 29.47185866157323 then False.
if x = 30.649896096089876 then True.
if x = 30.71820166083716 then False.
if x = -7.407662554463532 then False.
if x = 33.62184825686014 then False.
i

That is a ton of rules if we use the data from our prior section. We need to generalize ther rules.

In [30]:
X = []
Y = []
for i in L:
    X.append(i[0])
    Y.append(i[1])

In [31]:
def find_edges(tree, label, X, Y):
    X.sort()
    Y.sort()
    diagonals = [i for i in set(X).intersection(set(Y))]
    diagonals.sort()
    L = [classify(tree, label, [d,d]) for d in diagonals]
    low = L.index(False)
    min_x = X[low]
    min_y = Y[low]
    high = L[::-1].index(False)
    max_x = X[len(X)-1 - high]
    max_y = Y[len(Y)-1 - high]
    
    return (min_x, min_y), (max_x, max_y)

In [32]:
find_edges(tree, label, X, Y) # you can use this to create a new rule to classify a point.

((-54.85152762143782, -44.762290633342296),
 (60.585669645835154, 51.149877004687355))

In [33]:
pred_coord_bounds = find_edges(tree, label, X, Y)

In [34]:
pred_coord_bounds[0][0] #minx
pred_coord_bounds[0][1] #miny
pred_coord_bounds[1][0] #maxx
pred_coord_bounds[1][1] #maxy

51.149877004687355

In [35]:
abs(pred_coord_bounds[0][0])+abs(pred_coord_bounds[1][0]) #xspan
abs(pred_coord_bounds[0][1])+abs(pred_coord_bounds[1][1]) #yspan

95.91216763802964

In [36]:
import turtle as tl

def draw_bag():
    tl.shape('turtle')
    tl.color('blue')
    tl.pen(pencolor = 'blue', pensize = 5)
    tl.penup()
    tl.goto(-35,35)
    tl.pendown()
    tl.right(90) # angle of turn
    tl.forward(70) #length of line
    tl.left(90)
    tl.forward(70)
    tl.left(90)
    tl.forward(70)

def draw_bag_guess(pred_coord_bounds):
    chad = tl.Turtle() 
    chad.shape('turtle')
    chad.color('red')
    chad.pen(pencolor = 'red', pensize = 5)
    chad.penup()
    chad.goto(pred_coord_bounds[0][0],pred_coord_bounds[1][1])
    chad.pendown()
    chad.right(90) # angle of turn
    chad.forward(abs(pred_coord_bounds[0][1])+abs(pred_coord_bounds[1][1])) #length of line
    chad.left(90)
    chad.forward(abs(pred_coord_bounds[0][0])+abs(pred_coord_bounds[1][0]))
    chad.left(90)
    chad.forward(abs(pred_coord_bounds[0][1])+abs(pred_coord_bounds[1][1]))
    chad.left(90)
    chad.forward(abs(pred_coord_bounds[0][0])+abs(pred_coord_bounds[1][0]))
    
def plot_turtles(data):
    billy = tl.Turtle()
    billy.shape('turtle')
    billy.color('green')
    plot_list = escaped_turtles(data)
    for i in plot_list:
        billy.penup()
        billy.setposition(i[0], i[1])
        billy.stamp()

def plot_new_turtles(data):
    '''
    input a list of xy coords [[x,y]]
    '''
    jake = tl.Turtle()
    jake.shape('turtle')
    jake.color('black')
    plot_list = data
    for i in plot_list:
        jake.penup()
        jake.setposition(i[0], i[1])
        jake.stamp()

In [37]:
def new_classifier(tree, label, data, pred_point):
    X = []
    Y = []
    for i in data:
        X.append(i[0])
        Y.append(i[1])
    min_x = find_edges(tree, label, X, Y)[0][0]
    min_y = find_edges(tree, label, X, Y)[0][1]
    max_x = find_edges(tree, label, X, Y)[1][0]
    max_y = find_edges(tree, label, X, Y)[1][1]
    x = pred_point[0]
    y = pred_point[1]
    if (min_x < x < max_x) and (min_y < y < max_y): 
        return 'Inside', True
    else:
        return 'Outside', False
    

In [38]:
print(new_classifier(tree, label, L, [-5, 5]))
print(new_classifier(tree, label, L, [-36, 5]))#change points around to see if it's within or outside the bounds

('Inside', True)
('Inside', True)


In [ ]:
import time

if __name__ == '__main__':
    tl.setworldcoordinates(-70.,-70.,70.,70.)
#     tl.ht()
    tl.speed(10)
#     tl.tracer(0,0) #ensure update is uncomented if you'd like to see the paths 
    draw_bag()
    draw_bag_guess(pred_coord_bounds)
    time.sleep(3)
    plot_turtles(L)
    plot_new_turtles([[-5,5]])
    time.sleep(3)
    plot_new_turtles([[-36,5]])
    tl.mainloop()

We now have a new classifier that uses the edges found from our training data to determin the bounding box for our decision tree!